In [1]:
def Samples():
    
    import cv2
    import numpy as np

    face_classifier = cv2.CascadeClassifier('C:/Users/GOLU-5/Anaconda3/pkgs/libopencv-3.4.1-h875b8b8_3/Library/etc/haarcascades/haarcascade_frontalface_default.xml')


    def face_extractor(img):

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)

        if faces is():
            return None

        for(x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]

        return cropped_face


    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count+=1
            face = cv2.resize(face_extractor(frame),(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = 'C:/Users/GOLU-5/Documents/face_sample/user'+str(count)+'.jpg'
            cv2.imwrite(file_name_path,face)

            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Face Cropper',face)
        else:
            print("Face not Found")
            pass

        if cv2.waitKey(5)==13 or count==100:
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Colleting Samples Complete!!!')


In [2]:
def Detect():
    import cv2
    import numpy as np
    from os import listdir
    from os.path import isfile, join


    data_path='C:/Users/GOLU-5/Documents/face_sample/'
    only_files=[f for f in listdir(data_path) if isfile(join(data_path,f))]
    print(only_files)

    train_data, label = [], []
    # enumerate() provide number of iterations
    for i,files in enumerate(only_files):
        image_path= data_path+only_files[i]
        images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        train_data.append(np.asarray(images,dtype=np.uint64))
        label.append(i)

    label=np.asarray(label,dtype=np.int32)
    # Local Binary Patterns Histogram 
    model=cv2.face.LBPHFaceRecognizer_create()

    model.train(np.asarray(train_data),np.asarray(label))
    print("Model Training Complete !!!!")


    face_classifier = cv2.CascadeClassifier('C:/Users/GOLU-5/Anaconda3/pkgs/libopencv-3.4.1-h875b8b8_3/Library/etc/haarcascades/haarcascade_frontalface_default.xml')

    def face_detector(img,size=0.5):
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=face_classifier.detectMultiScale(gray,1.3,5)

        if faces is():
            return img,[]

        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            roi=img[x:x+w,y:y+h]
            roi=cv2.resize(roi,(200,200))

            return img,roi
    cap=cv2.VideoCapture(0)

    while True:
        ret,frame=cap.read()

        image,face=face_detector(frame)

        try:
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            result=model.predict(face)
            print(result)
            # checking confidence value

            if result[1]<500:
                confidence=int(100*(1-(result[1])/300))
                display_string=str(confidence)+' % Confident it is user'
                cv2.putText(image,display_string,(100,200),cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)

            if confidence>75:
                cv2.putText(image,"Unlocked",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.imshow("Face Cropper",image)

            else:
                cv2.putText(image,"Locked",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                cv2.imshow("Face Cropper",image)


        except:
            cv2.putText(image,"Face not fount",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            cv2.imshow("Face Cropper",image)
            pass

        if cv2.waitKey(1)==13:
            break


    cap.release()
    cv2.destroyAllWindows()


In [3]:
#import tkinter as tk
from tkinter import *
from tkinter import messagebox 
root=Tk()
root.title("FACE DETECTION AND RECOGNIZER")
root.geometry("500x500")
root.maxsize(500,500)
root.minsize(250,250)
root.configure(bg="lightblue")

def ask_quit():
    if messagebox.askokcancel("Quit", "You want to quit now?"):
        root.destroy()

collect=Label(root,text="sample collector",bg='lightblue')
collect.grid(row=0,column=0,padx=20,pady=10)

b1=Button(root,text="collect",command=Samples) #button
b1.grid(row=0,column=1,pady=30)

recognize=Label(root,text="Detect and Recogize",bg='lightblue')
recognize.grid(row=4,column=0,padx=70,pady=40)

b2=Button(root,text="Recognize",command=Detect) #detect button
b2.grid(row=4,column=1,pady=10)

b3=Button(root,text="Exit",command=ask_quit)
b3.grid(row=10,column=0,pady=30)

root.mainloop()